### Model 1 (Map if there is tendency of prices)


**Question(s) to Answer:**

- How will the prices per square meter and per disctricy be next quarter? (Fixed at 2009)

In [1]:
import pandas as pd
import os
import numpy as np
import glob

In [2]:
data = pd.read_excel('../input/BCRP-departamentos.xlsx')
data['Periodo'] = data['Año'].astype(str) + '_' + data['Trimestre'].astype(str)

In [13]:
data.reset_index(inplace=True)

**We will reshape the dataset into quarterly data grouped by district**